# Bevezetés a valószínűségszámítás és statisztika témakörébe
Ebben a jegyzetfüzetben néhány korábban megvitatott fogalommal fogunk játszani. A valószínűségszámítás és statisztika sok fogalma jól képviselteti magát a Python adatfeldolgozó nagy könyvtáraiban, mint például a `numpy` és a `pandas`.


In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt

## Véletlen változók és eloszlások
Kezdjük egy 30 értékből álló minta húzásával egy 0 és 9 közötti egyenletes eloszlásból. Kiszámoljuk a várható értéket és a varianciát is.


In [ ]:
sample = [ random.randint(0,10) for _ in range(30) ]
print(f"Sample: {sample}")
print(f"Mean = {np.mean(sample)}")
print(f"Variance = {np.var(sample)}")

Annak vizuális becsléséhez, hogy hány különböző érték található a mintában, ábrázolhatjuk a **hisztogramot**:


In [ ]:
plt.hist(sample)
plt.show()

## Valós adatok elemzése

Az átlag és a szórás nagyon fontosak a valós adatok elemzésekor. Töltsük be az adatokat baseball-játékosokról a [SOCR MLB Height/Weight Data](http://wiki.stat.ucla.edu/socr/index.php/SOCR_Data_MLB_HeightsWeights) oldalról.


In [ ]:
df = pd.read_csv("../../data/SOCR_MLB.tsv",sep='\t', header=None, names=['Name','Team','Role','Weight','Height','Age'])
df


> Egy olyan csomagot használunk itt, amely a [**Pandas**](https://pandas.pydata.org/) nevet viseli az adatelemzéshez. Ebben a tanfolyamban később még többet fogunk beszélni a Pandas-ról és az adattal való munkáról Pythonban.

Számoljuk ki az életkor, magasság és súly átlagértékeit:


In [ ]:
df[['Age','Height','Weight']].mean()

Most összpontosítsunk a magasságra, és számoljuk ki a szórást és a varianciát:


In [ ]:
print(list(df['Height'])[:20])

In [ ]:
mean = df['Height'].mean()
var = df['Height'].var()
std = df['Height'].std()
print(f"Mean = {mean}\nVariance = {var}\nStandard Deviation = {std}")

Az átlag mellett érdemes megvizsgálni a mediánt és a kvartiliseket is. Ezeket egy **dobozábra** segítségével lehet megjeleníteni:


In [ ]:
plt.figure(figsize=(10,2))
plt.boxplot(df['Height'].ffill(), vert=False, showmeans=True)
plt.grid(color='gray', linestyle='dotted')
plt.tight_layout()
plt.show()

Készíthetünk dobozdiagramokat az adataink részhalmazairól is, például játékosi szerep szerint csoportosítva.


In [ ]:
df.boxplot(column='Height', by='Role', figsize=(10,8))
plt.xticks(rotation='vertical')
plt.tight_layout()
plt.show()

> **Megjegyzés**: Ez az ábra azt sugallja, hogy átlagosan a első bázisznál játszók magassága nagyobb, mint a második bázisnál játszóké. Később megtanuljuk, hogyan tudjuk ezt a hipotézist formálisan tesztelni, és hogyan bizonyíthatjuk, hogy adataink statisztikailag szignifikánsak ennek alátámasztására.

Az életkor, a magasság és a testsúly mind folytonos valószínűségi változók. Ön szerint milyen eloszlásuk van? Egy jó módja, hogy kiderítsük, az, ha megrajzoljuk az értékek hisztogramját:


In [ ]:
df['Weight'].hist(bins=15, figsize=(10,6))
plt.suptitle('Weight distribution of MLB Players')
plt.xlabel('Weight')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

## Normális eloszlás

Hozzunk létre egy mesterséges súlymintát, amely normális eloszlást követ, és ugyanazzal az átlaggal és szórással rendelkezik, mint a valós adataink:


In [ ]:
generated = np.random.normal(mean, std, 1000)
generated[:20]

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(generated, bins=15)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
plt.hist(np.random.normal(0,1,50000), bins=300)
plt.tight_layout()
plt.show()

Mivel a valós életben a legtöbb érték normál eloszlású, nem szabad egyenletes eloszlású véletlenszám-generátort használnunk mintadatok előállításához. Íme, mi történik, ha megpróbálunk súlyokat generálni egyenletes eloszlással (amit a `np.random.rand` generál):


In [ ]:
wrong_sample = np.random.rand(1000)*2*std+mean-std
plt.figure(figsize=(10,6))
plt.hist(wrong_sample)
plt.tight_layout()
plt.show()

## Bizalmi intervallumok

Most számoljuk ki a baseball játékosok súlyára és magasságára vonatkozó bizalmi intervallumokat. A kódot [ebből a stackoverflow beszélgetésből](https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data) fogjuk használni:


In [ ]:
import scipy.stats

def mean_confidence_interval(data, confidence=0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a), scipy.stats.sem(a)
    h = se * scipy.stats.t.ppf((1 + confidence) / 2., n-1)
    return m, h

for p in [0.85, 0.9, 0.95]:
    m, h = mean_confidence_interval(df['Weight'].fillna(method='pad'),p)
    print(f"p={p:.2f}, mean = {m:.2f} ± {h:.2f}")

## Hipotézisvizsgálat

Vizsgáljuk meg a különböző szerepköröket a baseball játékosok adatállományában:


In [ ]:
df.groupby('Role').agg({ 'Weight' : 'mean', 'Height' : 'mean', 'Age' : 'count'}).rename(columns={ 'Age' : 'Count'})

Teszteljük azt a hipotézist, hogy az első bázis játékosok magasabbak, mint a második bázis játékosok. A legegyszerűbb módja ennek a konfidencia intervallumok tesztelése:


In [ ]:
for p in [0.85,0.9,0.95]:
    m1, h1 = mean_confidence_interval(df.loc[df['Role']=='First_Baseman',['Height']],p)
    m2, h2 = mean_confidence_interval(df.loc[df['Role']=='Second_Baseman',['Height']],p)
    print(f'Conf={p:.2f}, 1st basemen height: {m1-h1[0]:.2f}..{m1+h1[0]:.2f}, 2nd basemen height: {m2-h2[0]:.2f}..{m2+h2[0]:.2f}')

Láthatjuk, hogy az intervallumok nem fedik át egymást.

A hipotézist statisztikailag helyesebben bizonyítani egy **Student-féle t-próbával** lehet:


In [ ]:
from scipy.stats import ttest_ind

tval, pval = ttest_ind(df.loc[df['Role']=='First_Baseman',['Height']], df.loc[df['Role']=='Second_Baseman',['Height']],equal_var=False)
print(f"T-value = {tval[0]:.2f}\nP-value: {pval[0]}")

A `ttest_ind` függvény által visszaadott két érték a következő:
* a p-érték tekinthető annak a valószínűségnek, hogy a két eloszlásnak azonos az átlaga. Esetünkben nagyon alacsony, ami azt jelenti, hogy erős bizonyíték van arra, hogy az első alapemberek magasabbak.
* a t-érték a normalizált átlagkülönbség köztes értéke, amelyet a t-tesztben használnak, és egy adott megbízhatósági értékhez tartozó küszöbértékkel hasonlítanak össze.


## Normális eloszlás szimulálása a központi határeloszlás tételével

A Pythonban található pszeudo-véletlenszám generátor egyenletes eloszlást biztosít számunkra. Ha normális eloszlású generátort szeretnénk készíteni, használhatjuk a központi határeloszlás tételét. Egy normális eloszlású értékhez egyszerűen egy egyenletesen generált minta átlagát számoljuk ki.


In [ ]:
def normal_random(sample_size=100):
    sample = [random.uniform(0,1) for _ in range(sample_size) ]
    return sum(sample)/sample_size

sample = [normal_random() for _ in range(100)]
plt.figure(figsize=(10,6))
plt.hist(sample)
plt.tight_layout()
plt.show()

## Korreláció és a Gonosz Baseball Vállalat

A korreláció lehetővé teszi, hogy kapcsolatokat találjunk az adat-sorozatok között. Játékszintű példánkban képzeljük el, hogy létezik egy gonosz baseball vállalat, amely a játékosait a magasságuk szerint fizeti - minél magasabb a játékos, annál több pénzt kap. Tegyük fel, hogy az alapfizetés 1000 dollár, és ehhez jár egy 0-tól 100-ig terjedő bónusz, a magasság függvényében. A valós MLB-játékosokat vesszük alapul, és kiszámoljuk képzeletbeli fizetésüket:


In [ ]:
heights = df['Height'].fillna(method='pad')
salaries = 1000+(heights-heights.min())/(heights.max()-heights.mean())*100
print(list(zip(heights, salaries))[:10])

Most számoljuk ki ezeknek a sorozatoknak a kovarianciáját és korrelációját. A `np.cov` úgynevezett **kovariancia mátrixot** ad vissza, amely a kovariancia többszörös változóra való kiterjesztése. A kovariancia mátrix $M$ eleme, $M_{ij}$ az $X_i$ és $X_j$ bemeneti változók közötti kovariancia, míg a diagonális értékek, $M_{ii}$, az $X_i$ varianciáját jelentik. Hasonlóan, a `np.corrcoef` a **korrelációs mátrixot** adja vissza.


In [ ]:
print(f"Covariance matrix:\n{np.cov(heights, salaries)}")
print(f"Covariance = {np.cov(heights, salaries)[0,1]}")
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

A korreláció értéke 1 azt jelenti, hogy két változó között erős **lineáris kapcsolat** áll fenn. A lineáris kapcsolatot vizuálisan is láthatjuk, ha az egyik értéket a másikhoz ábrázoljuk:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights,salaries)
plt.tight_layout()
plt.show()

Lássuk, mi történik, ha a kapcsolat nem lineáris. Tegyük fel, hogy vállalatunk úgy döntött, elrejti a magasságok és fizetések közötti nyilvánvaló lineáris függőséget, és bevezet némi nemlinearitást a képletbe, például a `sin` függvényt:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

Ebben az esetben a korreláció kissé kisebb, de még mindig meglehetősen magas. Most, hogy a kapcsolat még kevésbé legyen nyilvánvaló, szeretnénk hozzáadni némi extra véletlenszerűséget úgy, hogy egy véletlenszerű változót adunk a fizetéshez. Nézzük, mi történik:


In [ ]:
salaries = 1000+np.sin((heights-heights.min())/(heights.max()-heights.mean()))*100+np.random.random(size=len(heights))*20-10
print(f"Correlation = {np.corrcoef(heights, salaries)[0,1]}")

In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(heights, salaries)
plt.tight_layout()
plt.show()

> Kitalálod, miért rendeződnek a pontok így függőleges vonalakká?

Megfigyeltük a mesterségesen létrehozott fogalom, azaz a fizetés és a megfigyelt változó, a *magasság* közötti összefüggést. Nézzük meg azt is, hogy a két megfigyelt változó, mint például a magasság és a súly, összefügg-e egymással:


In [ ]:
np.corrcoef(df['Height'].ffill(),df['Weight'])

Sajnos nem kaptunk eredményt - csak néhány furcsa `nan` értéket. Ennek oka, hogy a sorozatunk néhány értéke nem definiált, `nan`-ként van ábrázolva, ami az eredmény undefined jellegét is okozza. A mátrixot megnézve láthatjuk, hogy a problémás oszlop a `Weight`, mert a `Height` értékek önkorrelációját számoltuk ki.

> Ez a példa megmutatja az **adat-előkészítés** és a **tisztítás** fontosságát. Megfelelő adatok nélkül nem tudunk semmit kiszámolni.

Használjuk a `fillna` metódust a hiányzó értékek pótlására, és számoljuk ki a korrelációt: 


In [ ]:
np.corrcoef(df['Height'].fillna(method='pad'), df['Weight'])

Valóban van összefüggés, de nem olyan erős, mint mesterséges példánkban. Valóban, ha megnézzük az egyik érték a másikkal szembeni szórásdiagramját, a kapcsolat sokkal kevésbé lenne nyilvánvaló:


In [ ]:
plt.figure(figsize=(10,6))
plt.scatter(df['Weight'],df['Height'])
plt.xlabel('Weight')
plt.ylabel('Height')
plt.tight_layout()
plt.show()

## Következtetés

Ebben a jegyzetfüzetben megtanultuk, hogyan végezzünk alapműveleteket az adatokon statisztikai függvények kiszámításához. Most már tudjuk, hogyan használjuk a matematika és a statisztika egy megbízható eszköztárát ahhoz, hogy bizonyítsunk egyes hipotéziseket, és hogyan számítsuk ki a megbízhatósági intervallumokat tetszőleges változókra adott adatminta alapján.


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Tájékoztatás**:
Ezt a dokumentumot az AI fordító szolgáltatás [Co-op Translator](https://github.com/Azure/co-op-translator) segítségével fordítottuk le. Bár a pontosságra törekszünk, kérjük, vegye figyelembe, hogy az automatikus fordítások hibákat vagy pontatlanságokat tartalmazhatnak. Az eredeti dokumentum anyanyelvén tekintendő hiteles forrásnak. Kritikus információk esetén szakmai emberi fordítást javaslunk. Nem vállalunk felelősséget az ebből eredő félreértésekért vagy téves értelmezésekért.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
